# RAG Flow Testing Notebook

This notebook runs the main RAG flow in simple steps:
1) Start a new session (create user),
2) Upload a document (which triggers embedding on the backend),
3) Ask questions — both when there is no chat history (creates a new chat) and when there is an existing chat (reuses the same chat).

Run cells in order. Ensure the backend is running at http://127.0.0.1:8011

In [1]:
import requests
from pprint import pprint
import time

BASE_URL = "http://127.0.0.1:8011"

def print_response(r):
    print(f"Status: {r.status_code}")
    try: pprint(r.json())
    except: print(r.text)
    print('-'*60)

print('Setup complete. Base URL:', BASE_URL)

Setup complete. Base URL: http://127.0.0.1:8011


## 1) Start a new session (create user)

In [2]:
username = 'rag_test_user'
resp = requests.post(f'{BASE_URL}/users/', json={'username': username})
print_response(resp)

if resp.status_code in (200, 201):
    user_id = resp.json().get('id')
    print('Created user_id:', user_id)
else:
    raise SystemExit('Failed to create user - stop testing')


Status: 201
{'created_at': '2025-11-03T09:14:43.265881',
 'id': '2f36cd7d-24c4-47da-9ba6-c4025534eb53',
 'total_chats': 0}
------------------------------------------------------------
Created user_id: 2f36cd7d-24c4-47da-9ba6-c4025534eb53


## 2) Prepare and upload a document (triggers embedding on backend)

In [ ]:
import os
import requests

BASE_URL = "http://localhost:8011"
USER_ID = "2f36cd7d-24c4-47da-9ba6-c4025534eb53"

file_path = input("Enter file path (or press Enter to choose from current directory): ").strip()

if not file_path:
    files = [f for f in os.listdir('.') if os.path.isfile(f) and not f.endswith('.ipynb')]
    if not files:
        raise SystemExit("No files found in current directory. Please provide a valid file path.")

    print("\nAvailable files:")
    for i, name in enumerate(files, 1):
        print(f"{i}. {name} ({os.path.getsize(name):,} bytes)")

    choice = input("\nEnter file number or full path: ").strip()
    file_path = files[int(choice) - 1] if choice.isdigit() else choice

file_path = os.path.abspath(file_path)

if not os.path.exists(file_path):
    raise SystemExit(f"File not found: {file_path}")

print(f"\nSelected file: {file_path}")
print(f"Size: {os.path.getsize(file_path):,} bytes")

upload_url = f"{BASE_URL}/documents/upload"
print(f"\n⬆Uploading to: {upload_url}")

with open(file_path, "rb") as f:
    files = {"file": (os.path.basename(file_path), f)}
    data = {"user_id": USER_ID}
    response = requests.post(upload_url, files=files, data=data)

if response.status_code in (200, 201):
    doc_id = response.json().get("id") or response.json().get("document_id")
    print(f"Upload successful! Document ID: {doc_id}")
else:
    print(f"Upload failed (status {response.status_code})")
    print("Details:", response.text or "No details available.")



Available files:
1. file.pdf (907,936 bytes)

Selected file: c:\Users\BRT.MIBRAHIM\OneDrive - CMA CGM\fastapi_mini_project\scripts\file.pdf
Size: 907,936 bytes

⬆Uploading to: http://localhost:8011/documents/upload
Upload successful! Document ID: 7c617c73-92ae-465a-8831-15e12d3151fd


In [8]:
time.sleep(1)
docs_resp = requests.get(f'{BASE_URL}/documents', params={'user_id': user_id})
print("\nUser's documents:")
print_response(docs_resp)

if docs_resp.status_code == 200:
    docs = docs_resp.json()
    print(f"Found {len(docs)} document(s) for this user")
else:
    print("Could not retrieve documents list")


User's documents:
Status: 200
[{'document_id': '7c617c73-92ae-465a-8831-15e12d3151fd',
  'file_size': 907936,
  'file_type': '.pdf',
  'filename': 'file.pdf',
  'total_chunks': 3,
  'upload_date': '2025-11-03T09:16:19.445000'}]
------------------------------------------------------------
Found 1 document(s) for this user


## 3) Ask me anything — no previous chat (creates new chat)

In [9]:
import requests
import time

BASE_URL = "http://localhost:8011"
USER_ID = "2f36cd7d-24c4-47da-9ba6-c4025534eb53"

question = "What is the weather in Paris?"
print(f"You asked: {question}")

response = requests.post(
    f"{BASE_URL}/orchestrator/chat",
    json={"query": question, "user_id": USER_ID}
)

if response.status_code != 200:
    print(f"Failed (status {response.status_code}): {response.text}")
    raise SystemExit()

data = response.json()

chat_id = data.get("chat_id") or data.get("chatId") or data.get("id")

messages = data.get("messages", [])
bot_message = None
for m in messages:
    if m.get("userType") == "bot":
        bot_message = m.get("content")
        break

print(f"   (Using chat_id: {chat_id})")

if bot_message:
    print(f"\nAgent Answer:\n{bot_message}")
else:
    print("No bot response found.")

if chat_id:
    time.sleep(0.5)
    verify_resp = requests.get(f"{BASE_URL}/chat/message/chat/{chat_id}")

    if verify_resp.status_code == 200:
        messages_db = verify_resp.json()
        if messages_db:
            saved_question = messages_db[0].get("user_message", "")
            print(f"\n{'-'*80}")
            print(f"Message delivered to new chat")
            print(f"\nQuestion saved in database: {saved_question}")
            print(f"Chat ID: {chat_id}")
        else:
            print("\nNo messages found in database")
    else:
        print(f"\nCould not verify database save (status {verify_resp.status_code})")
else:
    print("\nWarning: No chat_id received from backend")
    print("Response data:", data)

You asked: What is the weather in Paris?
   (Using chat_id: 7eb3b3bb-a569-4707-b890-081ea0b31f37)

Agent Answer:
Hello! Here's the current weather in Paris:

It's about 11°C (51°F) right now, but it feels a bit cooler at 10°C (51°F). The sky is overcast with clouds. It's quite humid with 96% humidity, and there's a gentle breeze blowing at 15 km/h. Enjoy your day!

--------------------------------------------------------------------------------
Message delivered to new chat

Question saved in database: What is the weather in Paris?
Chat ID: 7eb3b3bb-a569-4707-b890-081ea0b31f37


## 4) Ask again — reuse existing chat (append to same chat)

In [10]:
import requests
import time

BASE_URL = "http://localhost:8011"
USER_ID = "2f36cd7d-24c4-47da-9ba6-c4025534eb53"
chat_id = "7eb3b3bb-a569-4707-b890-081ea0b31f37"

question = "what was my first message?"
print(f"You asked: {question}")
print(f"(Using chat_id: {chat_id})")

response = requests.post(
    f"{BASE_URL}/orchestrator/chat",
    params={"chat_id": chat_id},
    json={"query": question, "user_id": USER_ID}
)

if response.status_code != 200:
    print(f"Failed (status {response.status_code}): {response.text}")
    raise SystemExit()

data = response.json()
bot_message = next((m.get("content") for m in data.get("messages", []) if m.get("userType") == "bot"), None)

if bot_message:
    print(f"\nAgent Answer:\n{bot_message}")
else:
    print("No bot response found.")

time.sleep(0.5)
verify_resp = requests.get(f"{BASE_URL}/chat/message/chat/{chat_id}")

if verify_resp.status_code == 200:
    all_messages = verify_resp.json()
    if all_messages:
        latest_msg = all_messages[-1]
        saved_question = latest_msg.get("user_message", "")
        
        print(f"\n{'-'*80}")
        print(f"Message delivered to existing chat")
        print(f"\nQuestion saved in database: {saved_question}")
        print(f"Total messages in this chat: {len(all_messages)}")
    else:
        print("\nNo messages found in database")
else:
    print(f"\nCould not verify database save (status {verify_resp.status_code})")

You asked: what was my first message?
(Using chat_id: 7eb3b3bb-a569-4707-b890-081ea0b31f37)

Agent Answer:
Your first message was: "What is the weather in Paris?"

--------------------------------------------------------------------------------
Message delivered to existing chat

Question saved in database: what was my first message?
Total messages in this chat: 2
